Todo:
1、图号高亮 ok
2、图号定位段落 ok
3、说明书中"背景技术"、“发明内容”等缺失或重复   ok
4、更换图标 ok
5、正则表达式  ok
6、附图排序方法 ok
7、说明书标题重复判定条件 ok \r\n 还是存在问题，改为出现几次
8、加入分割线 ok
9、高亮图号 ok
10、滚动条 ok
11、支持快捷键 ok
12、图号中有空格的高亮 ok 选择的方法1
    两种方法1、去掉原始文本所有空格，简单粗暴  ok
            2、正匹配忽略空格，技术流，可以一试
13、防止最大化窗口以及拉伸 ok
14、支持鼠标右键粘贴等
15、增加查找功能
16、减少程序大小
17、加密
18、是否存在繁体字
19、是否存在？  ok
20、说明书信息检索模式 
21、获取标题 ok
22、增加判断 “未找到引用源” ok
23、增加段落数 ok
24、输出排版   ok
25、判断附图标号是否连续 ok
26、判断附图标记有字母的是否单有数字 ok
27、错误输出序号 ok
28、改名
29、相对大小




In [3]:
#################################################################
#author: xiaomililkx
#################################################################
from tkinter import *
import base64
import re
import os

LOG_LINE_NUM = 0

class MY_GUI():
    def __init__(self,init_window_name):
        self.init_window_name = init_window_name

    #设置窗口
    def set_init_window(self):
        self.init_window_name.title("Searcher   by: xiaomilifather")           #窗口名
        self.init_window_name.geometry('1100x700')
        #self.init_window_name["bg"] = "pink"                                    #窗口背景色
        #self.init_window_name.attributes("-alpha",0.9)                          #虚化，值越小虚化程度越高
        #标签
        self.init_data_label = Label(self.init_window_name, text="Ctrl+V 输入说明书")
        self.init_data_label.grid(row=0, column=0)
        self.result_data_label = Label(self.init_window_name, text="结果输出")
        self.result_data_label.grid(row=0, column=17)
        #文本框        
        self.init_data_Text = Text(self.init_window_name, width=70, height=50, undo = True)  #原始数据录入框
        #self.init_data_Text.grid(row=1, column=0, rowspan=10, columnspan=10)
        self.init_data_Text.grid(row=1, column=0)
        #列表框
        self.result_data_Listbox = Listbox(self.init_window_name, width=70, height=50)  #处理结果展示
        self.result_data_Listbox.grid(row=1, column=12, rowspan=15, columnspan=10)

        #纵向滚动条
        self.L_scroll=Scrollbar(self.init_window_name)
        self.L_scroll.grid(row=1, column= 10, sticky=S + N)#指定 上下展开
        self.L_scroll.config(command=self.init_data_Text.yview)
        self.init_data_Text.config(yscrollcommand=self.L_scroll.set)
        self.R_scroll=Scrollbar(self.init_window_name)
        self.R_scroll.grid(row=1, column= 22, sticky=S + N)#指定 上下展开
        self.R_scroll.config(command=self.result_data_Listbox.yview)
        self.result_data_Listbox.config(yscrollcommand=self.R_scroll.set)
        #按钮
        self.smsfind_button = Button(self.init_window_name, text="提交", bg="lightblue", width=10, command=self.sms_find)  # 调用内部方法  加()为直接调用
        self.smsfind_button.grid(row=1, column=11)
  
    #功能函数
    def sms_find(self):
        #初始化text（去空格）和Listbox（清空）
        ls = ['本申请信息如下：']
        flag = [] #存在的错误
        src = self.init_data_Text.get(1.0,END) 
        src = src.replace(r' ','')
        text_content = (src.replace(" ","")).split("\n")
        text_content.pop()#列表最后一个元素是空删除它
        #print(text_content)
        self.init_data_Text.delete(1.0, END)
        self.init_data_Text.insert(END, src)
        self.result_data_Listbox.delete(0, END)
        lspic, a = self.sms_pic(src) #附图查找
        etitle = self.sms_titles(src)#标题查找
        wh = self.sms_wh(src)#查找是否有？
        error1 = self.sms_error1(src)#查找是否有“引用源错误”
        if(a + etitle + wh + error1):
            flag = [r'==================分割线==================' ,'说明书存在以下错误：']
            flag   = flag + a + etitle + wh + error1
            for i in range(len(flag)-2):
                flag[i+2] = str(i+1) + '、' + flag[i+2]
        else:
            flag.append('说明书正常')
        print(flag)
        title  = self.sms_info(src)
        
        ls = ls + title + lspic + flag #所有结果列表汇总
        self.ptlist(ls)#输出


    #查找说明书格式标记\n",
    def sms_titles(self,src):
        lstitle = []
        pttitle = []
        if(src.count('技术领域') == 0):
            lstitle.append(r'没有“技术领域”')
        elif(src.count('技术领域') > 1):
            lstitle.append('出现'+str(src.count('技术领域')) +'次“技术领域”')
            pttitle.append(r'技术领域')
        else:
            pttitle.append(r'技术领域')

        if(src.count('背景技术') == 0):
            lstitle.append(r'没有“背景技术”')
        elif(src.count('背景技术') > 1):
            lstitle.append(r'出现'+str(src.count('背景技术')) +'次“背景技术”')
            pttitle.append(r'背景技术')
        else:
            pttitle.append(r'背景技术')

        if(src.count('发明内容') == 0):
            lstitle.append(r'没有“发明内容”')
        elif(src.count('发明内容') > 1):
            lstitle.append(r'出现'+str(src.count('发明内容')) +'次“发明内容”')
            pttitle.append(r'发明内容')
        else:
            pttitle.append(r'发明内容')

        if(src.count('附图说明') == 0):
            lstitle.append(r'没有“附图说明”')
        elif(src.count('附图说明') > 1):
            lstitle.append(r'出现'+str(src.count('附图说明')) +'次“附图说明”')
            pttitle.append(r'附图说明')
        else:
            pttitle.append(r'附图说明')

        if(src.count('具体实施方式') == 0):
            lstitle.append(r'没有“具体实施方式”')
        elif(src.count('具体实施方式') > 1):
            lstitle.append(r'出现'+str(src.count('具体实施方式')) +'次“具体实施方式”')
            pttitle.append(r'具体实施方式')
        else:
            pttitle.append(r'具体实施方式')
            
        self.sms_highlight(pttitle, 'title')#高亮
        return lstitle

    
    #图片标号查找                
    def sms_pic(self,scrp):
        rs = []#结果
        lspic = []
        nums = []
        flag = []
        sel = r'图\d+[a-zA-Z]?-?图?\d*[a-zA-Z]?' #可匹配  图11A-图14A
        #p = re.compile(sel, re.X)
        lspic = re.findall(sel, scrp) #正则匹配 “图+数字+字母”
        lspic = set(lspic) #去重
        lspic = sorted(lspic) #排序
        lspic.sort(key = lambda x:int(re.match('图(\d+)[a-zA-Z]?',x).group(1)))#排序
        if(lspic):
            rs.append('附图如下：')
            rs = rs + lspic
        else:
            rs.append('说明书不包含附图！')

        self.sms_highlight(lspic, 'pic')#高亮

        #查找是否缺少附图标号
        if(len(lspic) > 1):
            num = int(re.search('\d+', lspic[len(lspic)-1]).group())#获得最大值
            nums = set(re.findall(r'\d+', ''.join(lspic))) #拼接成字符串
            if(len(nums) != num):
                flag.append('附图图号不连续')
                
        #查找附图标号有字母且有数字
        sel = r'(图\d+)[a-zA-Z]'
        nums = set(re.findall(sel, ''.join(lspic))) #拼接成字符串
        #a = [x for x in list1 if x in list2] #两个列表中都存在
        #b = [y for y in (list1 + list2) if y not in a] #两个列表中的不同元素
        picnumerror = [x for x in nums if x in lspic]
        if(picnumerror): #两个列表中都存在
            st = '可能存在错误的图号：' +  ', '.join(sorted(picnumerror))
            flag.append(st)

        return rs, flag
    
    #查找是否有？并高亮
    def sms_wh(self,scrp):
        wh = []
        if(scrp.count('?' or '？') > 0):
            wh.append('具有' + str(scrp.count('?')+scrp.count('？'))  + '个问号')
        self.sms_highlight(['?','？'],'wh')
        return wh
        
    #查找是否有“未找到引用源”并高亮
    def sms_error1(self,scrp):
        error1 = []
        if(scrp.count('未找到引用源') > 0):
            error1.append('具有' + str(scrp.count('未找到引用源')) + r'个“未找到引用源”')
        self.sms_highlight(['未找到引用源'],'error1')
        return error1

    #发明名称，段落数
    def sms_info(self,scrp):
        title = ''
        info = []
        text_content = (scrp.replace(" ","")).split("\n")
        text_content.pop()#列表最后一个元素是空删除它
        #text_content = re.findall(r'*\n',scrp)
        #print(text_content)
        title = '发明名称：' + text_content[0]

        pa = ''
        tp = re.findall('\\[\d\d\d\d\\]', scrp)
        pa = '总计段落：'+ str(len(tp)) 
        info.append(title)
        info.append(pa)
        return info


    #高亮
    def sms_highlight(self, lspic,tagh):
        self.init_data_Text.tag_config('pic', background='yellow') #配置字体  图号
        self.init_data_Text.tag_config('title', background='pink') #配置字体  标题
        self.init_data_Text.tag_config('wh', background='red') #配置字体  出现问号
        self.init_data_Text.tag_config('error1', background='red') #配置字体 未找到引用源
        if (tagh == 'pic'):
            for i in range(len(lspic)):#循环显示图号高亮
                self.searchtag(self.init_data_Text, lspic[i], 'pic')
        elif(tagh == 'title'):
            for i in range(len(lspic)):#循环标题高亮
                self.searchtag(self.init_data_Text, lspic[i], 'title')
        elif(tagh == 'wh'):
            for i in range(len(lspic)):#循环问号高亮
                self.searchtag(self.init_data_Text, lspic[i], 'wh')
        elif(tagh == 'error1'):
            for i in range(len(lspic)):#循环未找到引用源高亮
                self.searchtag(self.init_data_Text, lspic[i], 'error1')


     #高亮查找字符串子函数
    def searchtag(self,text_widget, keyword, tag):
        pos = '1.0'
        while True:
            idx = text_widget.search(keyword, pos, END)
            if not idx:
                break
            pos = '{}+{}c'.format(idx, len(keyword))
            text_widget.tag_add(tag, idx, pos)

    #打印
    def ptlist(self,lspic):
        for n in range(len(lspic)):
            self.result_data_Listbox.insert(END,lspic[n]+'\n')
               
#创建图标
def icocreate():
    tmp = open("tmp.ico", "wb+")
    tmp.write(base64.b64decode('AAABAAEAICAAAAEAIACoEAAAFgAAACgAAAAgAAAAQAAAAAEAIAAAAAAAABAAABILAAASCwAAAAAAAAAAAAAAAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AwMC/wEBAf8AAAD/AwMD/yYmIf9aVk7/nZeL/7ewof+Qi3//SkhD/wQEA/8AAAD/AAAA/wAAAP8AAAD/AAAA/wEBAf8DAwL/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AQEB/wMDAv8AAAD/BwgH/09NRf+lnpH/39TD/+Tbyf+Pin//KSgk/w0ODP8tKyj/WFNM/4F8cf+Yk4b/o5yQ/6Kajv9VUUr/CAgI/wAAAP8DAwL/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wEBAf8DAgL/AAAA/zw6NP+xq53/4tjI/+zi0f+oopX/HBwa/w4ODP9oZFv/vLam/97Vxv/j2sr/49nK/+HXyP/f1cb/39XH/+Tay/+4sKH/QkA6/wAAAP8CAgL/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8BAQH/AgEB/wAAAP9vbGL/39bG/9vRwv/f1cX/dnFn/wAAAP9bV1D/1Mu8/+bczP/a0MH/08m7/9HIuv/RyLr/0ci6/9LJu//Sybv/0Mi6/9vSw//h2Mj/gHtx/wYGBf8AAAD/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AQEB/wEBAf8BAQH/iYR7/+bdzP/Qxrn/39XH/2xpX/8AAAD/jYl+/+rf0P/WzL7/0Me5/9HIu//RyLr/0ci7/9PKvP/Uy73/08q9/9HJu//RyLr/0Me6/9HIuv/m3c3/m5aL/wgIB/8AAAD/AQEB/wAAAP8AAAD/AAAA/wAAAP8CAQH/AAAA/4aCeP/m3Mz/zcS3/+DWx/+Mh3z/AAAA/4SAdv/n3s7/z8a5/9HIuv/Sybz/29HD/+Payf/i2cn/29LD/9fOv//Zz8H/4NbH/+Payf/c0sT/0sm7/83Et//k2sv/mpWK/wQEA/8AAAD/AQEA/wAAAP8AAAD/AwMC/wAAAP9iX1f/5dvL/8/Guf/Zz8H/xLyu/wUGBP9EQz3/5NrK/9DHuf/RyLr/3tTF/93VxP+0rqH/dnFp/0A+OP8hIB7/HBsa/yclIv9DQDv/dHBn/7StoP/d1cX/3dPF/8zEtv/l3Mz/fHdt/wAAAP8CAgL/AAAA/wICAv8AAAD/KCgk/9XMvv/Uy73/0Me6/+LZyf9bWVH/AAAA/7ixov/a0cH/0sm8/+LYyf+moJT/PDo0/wQFA/8CBAT/GhoY/ywqJ/8dHBn/AwMC/wAAAP8AAAD/AAAA/zk5NP+vqZ3/4tjJ/9DHuf/f1cb/QD04/wAAAP8DAgL/AgEB/wAAAP+UkYT/4dfH/9DHuf/Wzb//zsa3/w4NDP8zMi3/3NPD/9XMvv/Yz8D/Yl5W/wEBAf8AAAD/AgIC/wwNC/8jIiD/XFlR/62nmf/Rybn/p6KT/21sYv8uLSn/AAAA/wAAAP93c2v/39bG/9vSw/+vqZz/AwQE/wAAAP8AAAD/Ly4p/9rTwf/Tyrv/0cm7/9/Vxv+knpL/AAAA/3Vyaf/l28z/0ci5/z49Nv8AAAD/AwMD/wUFBf8DAwP/AAAB/wAAAP8AAAD/AgMC/2FeV//h2Mf/5tzM/9nQwf+uppn/QD03/wAAAP9iX1f/3dXF/9/Wx/9HRT//AAAA/wAAAP9eW1L/5NzL/9DHuv/RyLr/49nK/3VyaP8AAAD/p6CT/+fdzf9LSkP/AAAA/wUFBf8CAgL/AAAA/wAAAP8AAAD/AAAA/wICAv8EBAT/AAAA/z07Nv/RyLr/1cy+/9zSxP/i2cj/hYN3/wAAAP9zbmb/7uTU/5uUif8AAAD/AAAA/1ZUS//k2sr/0ci6/9HIuv/k2sr/WldO/wAAAP/Hwa//j4t+/wAAAP8EBAP/AQEB/wAAAP8uLi7/ZWNj/25tbP9MSkr/Dg0N/wAAAP8GBgX/AAAA/0pJQf/d08T/0ce6/9DIuv/o3c7/n5qP/wQFBP+rpZj/1su8/xgYFP8AAAD/SklC/+LZyP/RyLv/0ci6/+Tayv9WVEv/AAAA/763pv8lJSH/AAAA/wICAv8BAAD/bWxs/769vv/Bv8D/wL6//8PBwv+hn5//JCQj/wAAAP8FBQX/AAAA/4mEef/h18j/0ci6/87GuP/p38//fHhu/y8uKf/i2cf/REI9/wAAAP82NDD/3NPD/9LKvP/QyLr/5NrJ/2dkW/8HCAb/j4p9/wAAAP8DAwP/AAAA/1lYWP/HxMX/sq+x/7Kvsf+yr7H/sK2w/8C9vv+hn6D/CgsL/wAAAP8AAAD/GxsX/83Ftv/XzcD/08q8/9PIu//j2cn/MjIt/5uXiv98d23/MDAs/y4tKf/Nxbb/1sy//9HIuf/h2Mb/kIuA/wwNC/9YVk//AAAA/wAAAP8IBwf/o6Ch/7m2uP+zsLL/tbK0/7WytP+1srT/sa2w/8LAwf9JSUn/AAAA/wYGBf8AAAD/f3xw/+LZyP/RyLr/0ci7/+HXyf+RjID/RUM9/42Kfv9jX1n/MzIt/6mjl//d08X/0ci7/9nQwf+9tqj/GBgW/ywsKP8AAAD/AAAA/yMiIv+3tbb/tLGz/7WytP+1srT/tbK0/7WytP+yr7H/wb6//2VjY/8AAAD/AwMD/wAAAP87OjX/2tHB/9PKvP/Tyrz/1sy+/9HHuf8zMy3/bGxi/3RxZ/9jX1f/amde/+bcy//QyLr/0sm7/93Txf9BPzv/CAgJ/wICAv8AAAD/GBcX/7Cur/+1srX/tbK0/7WytP+1srT/tbK0/7Kvsf/Cv8D/XVtb/wAAAP8AAAD/GxoX/yYlIf/Auaz/2c/B/9PKvP/RyLv/4djJ/01LRP8nJyT/X1tU/7Gqm/8vMCv/29LD/9PJvP/Rybv/39bG/5uWif8AAAD/AgIC/wEBAf8AAAD/hoSF/8G+wP+wra//tLKz/7WytP+zsLL/s7Cy/7y6u/8qKir/AAAA/wAAAP9PTET/JCMf/6GakP/f1cf/0sm7/9HIuv/k2sv/c25l/wAAAP8zMS3/4djH/z89N/+MiHz/5NrL/9DHuf/Tyrz/3NPE/0NCPP8AAAD/BQUF/wAAAP8jIyP/rq2u/8K/wf+2s7X/tbK0/7q3uv/Gw8T/ZGNj/wAAAP8DAwP/AAAA/5CNgf8gIRz/jYh//+HYyf/RyLv/0ci7/+LYyf+NiXz/AAAA/xAQDv/Oxbf/rKWZ/x0dGv/Px7j/2dHB/8/Guf/a0ML/wrut/xUWE/8AAAD/BQUE/wAAAP8jIyP/goGB/6uqq/+zsbL/m5qa/09OT/8AAAD/AwMC/wAAAP8iIR3/yMOx/xARDv+OiX7/49jJ/9HIu//Rybv/4dfI/5eThv8AAAD/AAAA/5iRhf/x5tX/XltU/y0sKP/d1MT/3NPD/8zEtv/d1MX/rKWX/w8PDv8AAAD/BQQE/wIBAf8AAAD/EhMS/xwbG/8FBQX/AAAA/wICAv8DAwP/AAAA/5aShf/Rybj/AwMD/5qVi//h1sf/0cm7/9HJu//h18f/mZOI/wAAAP8AAAD/TEpD/+HZyP/b0cP/Pjs2/yQjH//Du6v/593N/9XMvv/d1MT/s62f/ygnI/8AAAD/AQEB/wAAAP8AAAD/AAAA/wEBAf8FBAT/BAQD/wAAAP9nY1v/6d/O/6ymmf8FBQT/uLCk/9vSw//Tyrz/0Mi6/+Tayv9uamH/AAAA/wAAAP8KCgn/urOl/9zSxP/Z0MD/QT85/wAAAP9qZ17/xLyu/9rRwv/r4dH/5NvK/46Kff83NC//EhEQ/wsLCv8KCgn/AwMD/wAAAP8AAAD/bWhf/9vSwv/j2cr/d3Nq/xQVEv/Vzb3/1cy+/9PKvP/VzL7/0cm6/x4eGv8AAAD/AwMD/wAAAP9VUkv/4tnI/9HIu//g1sf/eHVq/wMDAv8CAgL/LCkn/1VTTP9/e3H/oZyP/5iUh/9pZl7/MzQv/woLCf8AAAD/JSUh/6Cbjv/h2Mf/0ci6/+DWx/8tKyf/U1JK/+Tay//RyLv/z8e5/+Tay/93cmn/AAAA/wMDA/8AAAD/AAAA/wAAAP+gmo7/4NbH/87GuP/j2cr/xr6v/2dkW/8YGBb/AAAA/wAAAP8AAAD/AAAA/wAAAP8PDgz/Ozg0/4+Kf//UzL3/39bG/8/GuP/d1MX/r6mb/woKCf+6sqX/29LD/8/Huf/c08P/u7Wm/w8ODf8AAAD/AQEB/wAAAP8BAQH/AAAA/xoZFv/Du63/3NLE/83Et//Z0MH/5NrL/9fOwP+/uKr/qaOW/5+Yjf+im4//sqyd/8rBs//f1cf/4dfI/9XLvf/Qx7r/0sm7/+LZyf8tLCj/ZmFa/+Tay//Oxrj/183A/9bOvv8xMCv/AAAA/wICAv8AAAD/AAAA/wAAAP8CAgL/AAAA/ysqJv/Kw7T/3dPF/87GuP/RyLr/1Mu9/9rRwv/e1cX/4NbH/+DWxv/c08P/187A/9LJu//RyLv/0Mi6/9PKvP/r4dH/WlZP/zEvK//b0sP/0Mi6/9fOwP/a0ML/RUM+/wAAAP8DAwP/AAAA/wAAAP8AAAD/AAAA/wAAAP8DAwL/AAAA/ycmI/++tqj/49nK/9HIuv/RyLv/08q8/9HJu//RyLv/0ci7/9HJu//RyLv/0Mi6/9LJvP/h2Mj/3tXF/1VSS/80Mi3/0cq6/9PKvP/f1MX/0cm6/0E/Of8AAAD/AwMD/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8DAgL/AAAA/xQTEP+Ri4D/4djI/93Txf/Tyrz/08q8/9PKvf/VzL7/2M/A/9/Wx//n3Mz/3NPD/5yViv8zMy7/VlVN/9fOvv/e1Mb/4dnJ/6uklv8lJCH/AAAA/wMDA/8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8CAgL/AQEB/wAAAP8/PTj/qqSY/9vRwf/a0cL/2tDB/9TKu//Eu6//n5mN/2pnXv8/PTf/UlBJ/6ymmP/z6df/4tjI/7iypf9TUkv/BQUF/wAAAP8CAgL/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8BAQH/AgIC/wAAAP8GBgX/Kigk/zEvK/8rKib/Kyom/zk4M/9RUEn/gX1y/7Sun//Z0cH/xL2v/4uFev9APzj/CgoJ/wAAAP8CAgL/AgIC/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AwMD/wEBAf8AAAD/AAAA/wAAAP8EBAT/JiYi/0lIQv9RT0j/Ojgz/x0cGv8GBwb/AAAA/wAAAP8BAQH/AwMD/wEBAf8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA='))
    # tmp.write(base64.b64decode('粘贴icon.py字符串内容'))
    tmp.close()

def gui_start():
    init_window = Tk()              #实例化出一个父窗口
    init_window.resizable(width=False, height=False)#禁用拉伸
    icocreate() #创建图标
    init_window.iconbitmap("tmp.ico")
    os.remove("tmp.ico")  #删除icon文件
    ZMJ_PORTAL = MY_GUI(init_window)
    # 设置根窗口默认属性
    ZMJ_PORTAL.set_init_window()
    init_window.mainloop()          #父窗口进入事件循环，可以理解为保持窗口运行，否则界面不展示

gui_start()#启动

['==================分割线==================', '说明书存在以下错误：', '1、没有“技术领域”', '2、没有“背景技术”', '3、没有“发明内容”', '4、没有“附图说明”', '5、没有“具体实施方式”']


In [6]:
from tkinter import *
import webbrowser#进行网络的操作
root = Tk()

text = Text(root,width=30,height=5)
text.pack()

text.insert(INSERT,'hellodsfsdfsdfksgaudfydsfsdfsdfsdsd')

text.tag_add('tag1','1.7','1.12')
text.tag_config('tag1',background='yellow',foreground='red')

def show_arrow_cursor(event):
    text.config(cursor='arrow')

def show_xterm_cursor(event):
    text.config(cursor='xterm')

def click(event):
    webbrowser.open('http://www.fishc.com')
    

text.tag_bind('tag1','<Enter>',show_arrow_cursor)
text.tag_bind('tag1','<Leave>',show_xterm_cursor)
text.tag_bind('tag1','<Button-1>',click)

mainloop()

In [8]:
from tkinter import *
import webbrowser

class Application(Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.createWidget()

    def createWidget(self):
        self.w1 = Text(root,width =400,height=120,bg="gray")
        #宽度20个字母（10个汉字），高度一个行高
        self.w1.pack()
        Button(self,text="通过tag精确控制文本",command=self.testTag).pack(side="left")

    def testTag(self):
            self.w1.delete(1.0,END)
            self.w1.insert(INSERT,"good good good study,day day up!\nGui 学习\n python Gui \n 简书来Python搜一下")
            self.w1.tag_add("good",1.0,1.9)
            self.w1.tag_config("good",background="yellow",foreground="red")
            self.w1.tag_add("jianshu",4.0,4.4)
            self.w1.tag_config("jianshu",underline=True)
            self.w1.tag_bind("jianshu","<Button-1>",self.webshow)

    def webshow(self,even):
        w2 = webbrowser.open("http://www.jianshu.com")
        self.w1.insert(INSERT,w2)


if __name__=="__main__":
    root=Tk()
    root.geometry("600x330+200+300")
    app = Application(master=root)
    root.mainloop()


In [9]:
import tkinter as tk

t = tk.Text()
t.pack()
t.insert(tk.END, "Lorem ipsum dolor sit amet, consectetur adipiscing elit.")
t.tag_config("foo", background="yellow", foreground="red")
t.tag_add("foo", "1.6", "1.11")
t.tag_add("foo", "1.28", "1.39")

t.tag_config("sel_intersection", background="orange")

def sel_manager(event):
       t = event.widget
       tag_remove_all (t, "sel_intersection")

       f = map(TextIndex, t.tag_ranges("foo"))
       s = map(TextIndex, t.tag_ranges("sel"))

       if (len(s) == 0):
           return
       for f_start, f_end in zip(f[::2],f[1::2]):
           t.tag_add("sel_intersection", max(s[0],f_start), min(s[1], f_end))

def tag_remove_all(widget, tag_name):
       ranges =  map(str, t.tag_ranges(tag_name))
       for s, e in zip(ranges[::2], ranges[1::2]):
           widget.tag_remove(tag_name, s, e)

class TextIndex:
       '''needed for proper index comparison, ie "1.5" < "1.10"
       '''
       def __init__(self, textindex):
           self._str = str(textindex)
           self._l , self._c = map(int, self._str.split('.'))
       def __cmp__(self, other):
           cmp_l = cmp(self._l, other._l)
           if cmp_l !=0:
               return cmp_l
           else:
               return cmp(self._c, other._c)
       def __str__(self):
           return self._str

t.bind("<<Selection>>", sel_manager)
t.mainloop()

In [ ]:
这个牛逼，键盘输入自动高亮！

In [17]:
import re
import tkinter as tk

CODE = """#include <test.h>
#include "test.h"

for (int i = 0; i< 10;i++ ) {
    if (i < 3){
       int hello;
    }
}"""


def configure_tags(text_widget, tags):
    for tag, color in tags.items():
        text_widget.tag_delete(tag)
        text_widget.tag_config(tag, foreground=color)


def on_key_release(text_widget):
    lines = text_widget.get(1.0, tk.END).splitlines()
    regex = re.compile(
        r"(^\s*"
        r"(?P<if>if)" + "|"  # if condition
        r"(?P<for>for)" + "|"  # for loop
        r"(?P<include>#include\s+[\"<]\S+)" + "|"
        r"(?P<int>int)"  # variable
        r"[\s\(]+)"
    )
    for idx, line in enumerate(lines):
        int_tag = f"int_{idx}"
        for_tag = f"for_{idx}"
        if_tag = f"if_{idx}"
        include_tag = f"include_{idx}"
        tags = {
            int_tag: "blue",
            for_tag: "green",
            if_tag: "purple",
            include_tag: "green",
            # add new tag here
        }
        configure_tags(text_widget, tags)

        for match in regex.finditer(line):
            for tag in tags:
                group_name = tag.split("_")[0]
                if -1 != match.start(group_name):
                    text_widget.tag_add(
                        tag,
                        "{0}.{1}".format(idx+1, match.start(group_name)),
                        "{0}.{1}".format(idx+1, match.end(group_name))
                    )


def main():
    root = tk.Tk()

    text = tk.Text(root)
    text.grid()

    text.bind("<KeyRelease>", lambda event: on_key_release(text))
    text.bind("<Enter>", lambda event: on_key_release(text))

    text.insert(tk.END, CODE)

    root.mainloop()


if __name__ == "__main__":
    main()

In [5]:
# 记事本
__author__ = "Yang"

import os
import tkinter as tk
import tkinter.filedialog
import tkinter.messagebox
import hashlib
import time


''' 函数列表 '''


''' 点击触发新建 '''
def newBuild():
    global md5Str, nowOperateFilePath
    # 清空Text内容
    text.delete(index, tk.END)
    md5Str = md5("")
    nowOperateFilePath = ""
    # 设置窗口标题
    setWindowTitle()
    # 设置行列号
    setMarkPostion()


''' 将内容保存到文件'''
def saveFile(filePath):
    global nowOperateFilePath, isSaveStatus, md5Str
    # 去除Text文本内容最后的\n
    content = text.get(index, tk.END)[:-1]
    with open(filePath, "w+", encoding = "utf-8") as f:
        f.write(content)
        # 将内容保存为md5格式，用于对比内容是否发生改变
        md5Str = md5(content)
    # 保存当前操作的路径
    nowOperateFilePath = filePath
    # 设置当前文本已保存态
    isSaveStatus = True
    setWindowTitle(filePath)
    print("保存文件成功，路径：" + filePath)

''' 根据是否保存状态，返回一个字符'''
def returnTitle():
    global isSaveStatus
    if isSaveStatus is True:
        return ''
    return '*'

''' 设置窗口标题'''
def setWindowTitle(filePath = None):
    if filePath is None or filePath == '':
        window.title("无标题* - 记事本")
    else:
        fileName = os.path.basename(filePath)
        window.title("%s%s - 记事本" % (fileName, returnTitle()))

''' 保存文件 '''
def toSaveFile():
    global nowOperateFilePath
    if nowOperateFilePath == "":
        # 文件不存在，新建
        try: 
            filePath = tk.filedialog.asksaveasfilename(title = u"保存", filetypes = fileTypes, initialdir = initDir)
            if filePath is not None:
                saveFile(filePath)
            else:
                print("保存文件出错")
        except Exception  as e:
            print(e)
    else:
        # 文件存在，直接保存
        saveFile(nowOperateFilePath)
        
    
''' 打开文件 '''
def toOpenFile():
    global nowOperateFilePath, md5Str
    try:
        filePath = tk.filedialog.askopenfilename(title = u"打开", filetypes = fileTypes, initialdir = initDir)
        if filePath is not None:
           with open(filePath, "r", encoding = "utf-8") as f:
               newBuild()
               content = f.read()
               text.insert(tk.INSERT, content)
               md5Str = md5(content)
           print("打开文件成功，路径：" + filePath)
           nowOperateFilePath = filePath
           # 设置窗口标题
           setWindowTitle(filePath)
           setMarkPostion(None, tk.INSERT)
        else:
            print("打开文件出错")
    except Exception as e:
        print(e)

    
''' MD5加密'''
def md5(content):
    md5 = hashlib.md5()
    md5.update(str(content).encode("utf-8"))
    return md5.hexdigest()

''' 另存为 '''
def toSaveOtherFile():
    try:
        file = tk.filedialog.asksaveasfile(title = u"另存为", filetypes = fileTypes, initialdir = initDir)
        if file is not None:
            saveFile(file.name)
            print("完成另存为")
        else:
            print("另存为出错")
    except Exception as e:
        print(e)

''' 获取光标所在的行和列 '''
def getIndex(text, index):
    return tuple(map(int, str.split(text.index(index), ".")))

''' 关闭主窗口触发'''
def callCloseWindow():
    if isSaveStatus is False:
        # 弹窗提示
        okCancel = tk.messagebox.askokcancel("提示", "内容没有保存，是否直接关闭？")
        if okCancel is True:
            window.destroy()
        else:
            print("cancel")
    else:
        window.destroy()

''' 剪切'''
def toCut():
    text.event_generate("<<Cut>>")
    
''' 复制'''
def toCopy():
    text.event_generate("<<Copy>>")

''' 粘贴'''
def toPaste():
    text.event_generate("<<Paste>>")
    # 设置光标位置
    setMarkPostion()

''' 撤销'''
def toUndo():
    text.event_generate("<<Undo>>")

''' 恢复撤销'''
def toRedo():
    text.event_generate("<<Redo>>")

''' 关闭查找框回调'''
def callCloseFindDialog(findDialog):
    # 删除标签，关闭搜索窗口
    removeTag(index)
    findDialog.destroy()

''' 删除标签'''
def removeTag(startIndex, endIndex = tk.END, tagName = "find"):
    text.tag_remove(tagName, startIndex, endIndex)

''' 开始搜索'''
def findTextStr(strs):
    startIndex = 1.0
    # 删除标签
    removeTag(startIndex)
    if strs:
        # 记录找到的个数
        count = 0
        while True:
            startIndex = text.search(strs, startIndex, nocase = True, stopindex = tk.END)
            if not startIndex:
                break
            lastIndex = "%s+%sc" % (startIndex, len(strs))
            text.tag_add("find", startIndex, lastIndex)
            startIndex = lastIndex
            count += 1
        text.tag_config("find", background = "#0088A8", foreground = "#fff")
        if count == 0:
            tk.messagebox.showwarning(title = "提示", message = "没有找到内容")

''' 动态设置搜索按钮状态'''
def setBtnStatus(fBtn, strs):
    if not strs:
        fBtn.config(state = tk.DISABLED)
    else:
        fBtn.config(state = tk.NORMAL)
    
''' 查找弹出框'''
def toFind():
    findDialog = tk.Toplevel(padx = 10, pady = 10)
    findDialog.title("查找")
    findDialog.resizable(0, 0)
    fW = 300
    fH = 50
    fX = int((screen_width - fW) / 2)
    fY = int((screen_height - fW) / 2)
    findDialog.geometry("%sx%s+%s+%s" %(fW, fH, fX, fY))
    
    tk.Label(findDialog, text = "查找：").grid(row = 0, column=0)
    fVar = tk.StringVar()
    # 输入框
    fEntry = tk.Entry(findDialog, textvariable = fVar, insertwidth = 1)
    fEntry.focus_set()
    fEntry.grid(row = 0, column=1)
    
    fBtn = tk.Button(findDialog, text = "查找所有", height = 1, state = tk.DISABLED, command = lambda : findTextStr(fVar.get()))
    fBtn.grid(row = 0, column = 3, ipadx = 8)
    
    fEntry.bind("<KeyRelease>", lambda event: setBtnStatus(fBtn, fVar.get()))
    
    tk.Label(findDialog, text = "  ").grid(row = 0, column=2)
   
    findDialog.grab_set()
    findDialog.focus_set()
    findDialog.protocol("WM_DELETE_WINDOW", lambda : callCloseFindDialog(findDialog))
    findDialog.mainloop()


''' md5判断内容是否改变 '''
def textIsChange():
    content = text.get(index, tk.END)
    if md5(content[:-1]) == md5Str:
        return False
    return True

''' 动态设置光标所在行和列'''
def setMarkPostion(index = None, type_ = tk.CURRENT):
    if index is None:
        line_num = getIndex(text, text.index(type_))
        var.set(["第%s行" % line_num[0], "第%s列" % (line_num[1] + 1)])
    else:
        text.focus()
        text.mark_set(type_,index)

''' 键盘输入事件'''
def keyBoard():
    global isSaveStatus
    # 设置光标的行和列
    setMarkPostion(None, tk.INSERT)
    # 判断内容是否发生改变
    isChange = textIsChange()
    if isChange is True:
        # 内容发生改变
        isSaveStatus = False
    else:
        # 内容没有发生改变
        isSaveStatus = True
    # 设置窗口标题
    setWindowTitle(nowOperateFilePath)


''' 窗口初始化配置 '''
window = tk.Tk()
win_width = 600
win_height = 500
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()
x = int((screen_width - win_width) / 2)
y = int((screen_height - win_height) / 2)

window.title("无标题* - 记事本")
window.geometry("%sx%s+%s+%s" % (win_width, win_height, x, y))

''' 全局变量'''
# 保存文件类型
fileTypes = [('text files', '.txt'), ('all files', '.*')]
#当前操作的文件路径
nowOperateFilePath = ""
# 文件是否保存状态
isSaveStatus = True
# 当前文件路径
initDir = os.path.abspath("./")
# Text 1行1列
index = 1.0
# Text 字体大小
textFont = 13
''' md5加密后的内容, 初始化md5内容'''
md5Str = md5("")

# 最好用绝对路径
#window.iconbitmap(initDir + "/icon.ico")


''' 设置菜单 '''
menu_bar = tk.Menu(window)
window.config(menu = menu_bar)

file_bar = tk.Menu(menu_bar, tearoff = 0)
file_bar.add_command(label = "新建", accelerator = "Ctrl + N", command = newBuild)
file_bar.add_command(label = "打开", accelerator = "Ctrl + O", command = toOpenFile)
file_bar.add_command(label = "保存", accelerator = "Ctrl + S", command = toSaveFile)
file_bar.add_command(label = "另存为", command = toSaveOtherFile)
file_bar.add_separator()
file_bar.add_command(label = "退出", command = window.destroy)
menu_bar.add_cascade(label = "文件", menu = file_bar)

edit_bar = tk.Menu(menu_bar, tearoff = 0)
edit_bar.add_command(label="撤销", accelerator="Ctrl + Z", command = toUndo)
edit_bar.add_command(label="回复撤销", accelerator="Ctrl + Y", command = toRedo)
edit_bar.add_separator()
edit_bar.add_command(label = "剪切", accelerator = "Ctrl + X", command = toCut)
edit_bar.add_command(label = "复制", accelerator = "Ctrl + C", command = toCopy)
edit_bar.add_command(label = "粘贴", accelerator = "Ctrl + V", command = toPaste)
edit_bar.add_separator()
edit_bar.add_command(label="查找", accelerator="Ctrl + F", command = toFind)
menu_bar.add_cascade(label = "编辑", menu = edit_bar)

about_menu = tk.Menu(menu_bar, tearoff = 0)
about_menu.add_command(label = "版本", accelerator = "v1.0.0")
about_menu.add_command(label = "作者", accelerator = "样子")
menu_bar.add_cascade(label = "关于", menu = about_menu)

''' 设置纵向滚动条 '''
scroll_bar = tk.Scrollbar(window)
scroll_bar.pack(fill = "y", side = "right")


''' 设置Text '''
text = tk.Text(window, font = ("宋体", textFont), padx = 2, pady = 2, undo = True, yscrollcommand = scroll_bar.set)
text.insert("insert", "")
# 设置光标在首行
setMarkPostion(index, "start")
text.pack(expand = 1, fill = "both")
scroll_bar.config(command = text.yview)

text.bind("<Control-Key-X>", lambda event: toCut())
text.bind("<Control-Key-C>", lambda event: toCopy())
text.bind("<Control-Key-V>", lambda event: toPaste())
text.bind("<Control-Key-Z>", lambda event: toUndo())
text.bind("<Control-Key-Y>", lambda event: toRedo())
text.bind("<Control-Key-f>", lambda event: toFind())

text.bind("<Control-Key-n>", lambda event: newBuild())
text.bind("<Control-Key-N>", lambda event: newBuild())
text.bind("<Control-Key-s>", lambda event: toSaveFile())
text.bind("<Control-Key-S>", lambda event: toSaveFile())
text.bind("<Control-Key-o>", lambda event: toOpenFile())
text.bind("<Control-Key-O>", lambda event: toOpenFile())
text.bind("<KeyRelease>", lambda event: keyBoard())
text.bind("<ButtonRelease-1>", lambda event: setMarkPostion())

''' 设置显示当前行 '''
var = tk.StringVar()
label = tk.Label(window, text = "第 1 行", height = 1, pady = 10, font = ("宋体", 10), textvariable=var)
setMarkPostion()
label.pack(side = "bottom", anchor = "w")


# 监听window关闭
window.protocol("WM_DELETE_WINDOW", callCloseWindow)
window.mainloop()

In [5]:
import tkinter as tk
import base64
import os

window = tk.Tk()
tmp = open("tmp.ico", "wb+")
tmp.write(base64.b64decode('AAABAAEAICAAAAEAIACoEAAAFgAAACgAAAAgAAAAQAAAAAEAIAAAAAAAABAAABILAAASCwAAAAAAAAAAAAAAAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AwMC/wEBAf8AAAD/AwMD/yYmIf9aVk7/nZeL/7ewof+Qi3//SkhD/wQEA/8AAAD/AAAA/wAAAP8AAAD/AAAA/wEBAf8DAwL/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AQEB/wMDAv8AAAD/BwgH/09NRf+lnpH/39TD/+Tbyf+Pin//KSgk/w0ODP8tKyj/WFNM/4F8cf+Yk4b/o5yQ/6Kajv9VUUr/CAgI/wAAAP8DAwL/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wEBAf8DAgL/AAAA/zw6NP+xq53/4tjI/+zi0f+oopX/HBwa/w4ODP9oZFv/vLam/97Vxv/j2sr/49nK/+HXyP/f1cb/39XH/+Tay/+4sKH/QkA6/wAAAP8CAgL/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8BAQH/AgEB/wAAAP9vbGL/39bG/9vRwv/f1cX/dnFn/wAAAP9bV1D/1Mu8/+bczP/a0MH/08m7/9HIuv/RyLr/0ci6/9LJu//Sybv/0Mi6/9vSw//h2Mj/gHtx/wYGBf8AAAD/AQEB/wAAAP8AAAD/AAAA/wAAAP8AAAD/AQEB/wEBAf8BAQH/iYR7/+bdzP/Qxrn/39XH/2xpX/8AAAD/jYl+/+rf0P/WzL7/0Me5/9HIu//RyLr/0ci7/9PKvP/Uy73/08q9/9HJu//RyLr/0Me6/9HIuv/m3c3/m5aL/wgIB/8AAAD/AQEB/wAAAP8AAAD/AAAA/wAAAP8CAQH/AAAA/4aCeP/m3Mz/zcS3/+DWx/+Mh3z/AAAA/4SAdv/n3s7/z8a5/9HIuv/Sybz/29HD/+Payf/i2cn/29LD/9fOv//Zz8H/4NbH/+Payf/c0sT/0sm7/83Et//k2sv/mpWK/wQEA/8AAAD/AQEA/wAAAP8AAAD/AwMC/wAAAP9iX1f/5dvL/8/Guf/Zz8H/xLyu/wUGBP9EQz3/5NrK/9DHuf/RyLr/3tTF/93VxP+0rqH/dnFp/0A+OP8hIB7/HBsa/yclIv9DQDv/dHBn/7StoP/d1cX/3dPF/8zEtv/l3Mz/fHdt/wAAAP8CAgL/AAAA/wICAv8AAAD/KCgk/9XMvv/Uy73/0Me6/+LZyf9bWVH/AAAA/7ixov/a0cH/0sm8/+LYyf+moJT/PDo0/wQFA/8CBAT/GhoY/ywqJ/8dHBn/AwMC/wAAAP8AAAD/AAAA/zk5NP+vqZ3/4tjJ/9DHuf/f1cb/QD04/wAAAP8DAgL/AgEB/wAAAP+UkYT/4dfH/9DHuf/Wzb//zsa3/w4NDP8zMi3/3NPD/9XMvv/Yz8D/Yl5W/wEBAf8AAAD/AgIC/wwNC/8jIiD/XFlR/62nmf/Rybn/p6KT/21sYv8uLSn/AAAA/wAAAP93c2v/39bG/9vSw/+vqZz/AwQE/wAAAP8AAAD/Ly4p/9rTwf/Tyrv/0cm7/9/Vxv+knpL/AAAA/3Vyaf/l28z/0ci5/z49Nv8AAAD/AwMD/wUFBf8DAwP/AAAB/wAAAP8AAAD/AgMC/2FeV//h2Mf/5tzM/9nQwf+uppn/QD03/wAAAP9iX1f/3dXF/9/Wx/9HRT//AAAA/wAAAP9eW1L/5NzL/9DHuv/RyLr/49nK/3VyaP8AAAD/p6CT/+fdzf9LSkP/AAAA/wUFBf8CAgL/AAAA/wAAAP8AAAD/AAAA/wICAv8EBAT/AAAA/z07Nv/RyLr/1cy+/9zSxP/i2cj/hYN3/wAAAP9zbmb/7uTU/5uUif8AAAD/AAAA/1ZUS//k2sr/0ci6/9HIuv/k2sr/WldO/wAAAP/Hwa//j4t+/wAAAP8EBAP/AQEB/wAAAP8uLi7/ZWNj/25tbP9MSkr/Dg0N/wAAAP8GBgX/AAAA/0pJQf/d08T/0ce6/9DIuv/o3c7/n5qP/wQFBP+rpZj/1su8/xgYFP8AAAD/SklC/+LZyP/RyLv/0ci6/+Tayv9WVEv/AAAA/763pv8lJSH/AAAA/wICAv8BAAD/bWxs/769vv/Bv8D/wL6//8PBwv+hn5//JCQj/wAAAP8FBQX/AAAA/4mEef/h18j/0ci6/87GuP/p38//fHhu/y8uKf/i2cf/REI9/wAAAP82NDD/3NPD/9LKvP/QyLr/5NrJ/2dkW/8HCAb/j4p9/wAAAP8DAwP/AAAA/1lYWP/HxMX/sq+x/7Kvsf+yr7H/sK2w/8C9vv+hn6D/CgsL/wAAAP8AAAD/GxsX/83Ftv/XzcD/08q8/9PIu//j2cn/MjIt/5uXiv98d23/MDAs/y4tKf/Nxbb/1sy//9HIuf/h2Mb/kIuA/wwNC/9YVk//AAAA/wAAAP8IBwf/o6Ch/7m2uP+zsLL/tbK0/7WytP+1srT/sa2w/8LAwf9JSUn/AAAA/wYGBf8AAAD/f3xw/+LZyP/RyLr/0ci7/+HXyf+RjID/RUM9/42Kfv9jX1n/MzIt/6mjl//d08X/0ci7/9nQwf+9tqj/GBgW/ywsKP8AAAD/AAAA/yMiIv+3tbb/tLGz/7WytP+1srT/tbK0/7WytP+yr7H/wb6//2VjY/8AAAD/AwMD/wAAAP87OjX/2tHB/9PKvP/Tyrz/1sy+/9HHuf8zMy3/bGxi/3RxZ/9jX1f/amde/+bcy//QyLr/0sm7/93Txf9BPzv/CAgJ/wICAv8AAAD/GBcX/7Cur/+1srX/tbK0/7WytP+1srT/tbK0/7Kvsf/Cv8D/XVtb/wAAAP8AAAD/GxoX/yYlIf/Auaz/2c/B/9PKvP/RyLv/4djJ/01LRP8nJyT/X1tU/7Gqm/8vMCv/29LD/9PJvP/Rybv/39bG/5uWif8AAAD/AgIC/wEBAf8AAAD/hoSF/8G+wP+wra//tLKz/7WytP+zsLL/s7Cy/7y6u/8qKir/AAAA/wAAAP9PTET/JCMf/6GakP/f1cf/0sm7/9HIuv/k2sv/c25l/wAAAP8zMS3/4djH/z89N/+MiHz/5NrL/9DHuf/Tyrz/3NPE/0NCPP8AAAD/BQUF/wAAAP8jIyP/rq2u/8K/wf+2s7X/tbK0/7q3uv/Gw8T/ZGNj/wAAAP8DAwP/AAAA/5CNgf8gIRz/jYh//+HYyf/RyLv/0ci7/+LYyf+NiXz/AAAA/xAQDv/Oxbf/rKWZ/x0dGv/Px7j/2dHB/8/Guf/a0ML/wrut/xUWE/8AAAD/BQUE/wAAAP8jIyP/goGB/6uqq/+zsbL/m5qa/09OT/8AAAD/AwMC/wAAAP8iIR3/yMOx/xARDv+OiX7/49jJ/9HIu//Rybv/4dfI/5eThv8AAAD/AAAA/5iRhf/x5tX/XltU/y0sKP/d1MT/3NPD/8zEtv/d1MX/rKWX/w8PDv8AAAD/BQQE/wIBAf8AAAD/EhMS/xwbG/8FBQX/AAAA/wICAv8DAwP/AAAA/5aShf/Rybj/AwMD/5qVi//h1sf/0cm7/9HJu//h18f/mZOI/wAAAP8AAAD/TEpD/+HZyP/b0cP/Pjs2/yQjH//Du6v/593N/9XMvv/d1MT/s62f/ygnI/8AAAD/AQEB/wAAAP8AAAD/AAAA/wEBAf8FBAT/BAQD/wAAAP9nY1v/6d/O/6ymmf8FBQT/uLCk/9vSw//Tyrz/0Mi6/+Tayv9uamH/AAAA/wAAAP8KCgn/urOl/9zSxP/Z0MD/QT85/wAAAP9qZ17/xLyu/9rRwv/r4dH/5NvK/46Kff83NC//EhEQ/wsLCv8KCgn/AwMD/wAAAP8AAAD/bWhf/9vSwv/j2cr/d3Nq/xQVEv/Vzb3/1cy+/9PKvP/VzL7/0cm6/x4eGv8AAAD/AwMD/wAAAP9VUkv/4tnI/9HIu//g1sf/eHVq/wMDAv8CAgL/LCkn/1VTTP9/e3H/oZyP/5iUh/9pZl7/MzQv/woLCf8AAAD/JSUh/6Cbjv/h2Mf/0ci6/+DWx/8tKyf/U1JK/+Tay//RyLv/z8e5/+Tay/93cmn/AAAA/wMDA/8AAAD/AAAA/wAAAP+gmo7/4NbH/87GuP/j2cr/xr6v/2dkW/8YGBb/AAAA/wAAAP8AAAD/AAAA/wAAAP8PDgz/Ozg0/4+Kf//UzL3/39bG/8/GuP/d1MX/r6mb/woKCf+6sqX/29LD/8/Huf/c08P/u7Wm/w8ODf8AAAD/AQEB/wAAAP8BAQH/AAAA/xoZFv/Du63/3NLE/83Et//Z0MH/5NrL/9fOwP+/uKr/qaOW/5+Yjf+im4//sqyd/8rBs//f1cf/4dfI/9XLvf/Qx7r/0sm7/+LZyf8tLCj/ZmFa/+Tay//Oxrj/183A/9bOvv8xMCv/AAAA/wICAv8AAAD/AAAA/wAAAP8CAgL/AAAA/ysqJv/Kw7T/3dPF/87GuP/RyLr/1Mu9/9rRwv/e1cX/4NbH/+DWxv/c08P/187A/9LJu//RyLv/0Mi6/9PKvP/r4dH/WlZP/zEvK//b0sP/0Mi6/9fOwP/a0ML/RUM+/wAAAP8DAwP/AAAA/wAAAP8AAAD/AAAA/wAAAP8DAwL/AAAA/ycmI/++tqj/49nK/9HIuv/RyLv/08q8/9HJu//RyLv/0ci7/9HJu//RyLv/0Mi6/9LJvP/h2Mj/3tXF/1VSS/80Mi3/0cq6/9PKvP/f1MX/0cm6/0E/Of8AAAD/AwMD/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8DAgL/AAAA/xQTEP+Ri4D/4djI/93Txf/Tyrz/08q8/9PKvf/VzL7/2M/A/9/Wx//n3Mz/3NPD/5yViv8zMy7/VlVN/9fOvv/e1Mb/4dnJ/6uklv8lJCH/AAAA/wMDA/8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8CAgL/AQEB/wAAAP8/PTj/qqSY/9vRwf/a0cL/2tDB/9TKu//Eu6//n5mN/2pnXv8/PTf/UlBJ/6ymmP/z6df/4tjI/7iypf9TUkv/BQUF/wAAAP8CAgL/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8BAQH/AgIC/wAAAP8GBgX/Kigk/zEvK/8rKib/Kyom/zk4M/9RUEn/gX1y/7Sun//Z0cH/xL2v/4uFev9APzj/CgoJ/wAAAP8CAgL/AgIC/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AwMD/wEBAf8AAAD/AAAA/wAAAP8EBAT/JiYi/0lIQv9RT0j/Ojgz/x0cGv8GBwb/AAAA/wAAAP8BAQH/AwMD/wEBAf8AAAD/AAAA/wAAAP8AAAD/AAAA/wAAAP8AAAD/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA='))
# tmp.write(base64.b64decode('粘贴icon2.py字符串内容'))
tmp.close()
window.title('窗口标题')
window.geometry('300x300')
window.iconbitmap("tmp.ico")
os.remove("tmp.ico")  #删除icon文件

window.mainloop()

In [30]:
import tkinter as tk 
from tkinter import * 
from tkinter import scrolledtext 
from tkinter import ttk 
# root = tk.Tk()
# monty = ttk.LabelFrame(root, text=' Monty Python') # 创建一个容器，其父容器为win
# monty.grid(column=0, row=0, padx=10, pady=10)
# scr = scrolledtext.ScrolledText(monty, width=30, height=5, wrap=tk.WORD)
# scr.grid(column=0, columnspan=3)
# root.mainloop()

root = tk.Tk() 
root.grid() 
app = ttk.Frame(root) 
app.grid()

fram1 = tk.LabelFrame(app, text='1') 
txt1 = tk.Text(fram1) 
sl1 = Scrollbar(fram1) 
sl1['command'] = txt1.yview
# sl1.grid(row=0, column=1,sticky=S + W  )
# txt1.grid(row=0, column=0,sticky=S + W  )
# fram1.grid(row=0, column=0, sticky=S + W )
sl1.grid(row=0, column=1, sticky=S + W + E + N)
txt1.grid(row=0, column=0, sticky=S + W + E + N)
fram1.grid(row=0, column=0, sticky=S + W + E + N)
mainloop()

In [13]:
from tkinter import *

root = Tk()
t = Text(root)
t.pack()
t.insert(END, '''\
blah blah blah Failed blah blah
blah blah blah Passed blah blah
blah blah blah Failed blah blah
blah blah blah Failed blah blah
''')
t.tag_config('failed', background='red')
t.tag_config('passed', background='blue')

def search(text_widget, keyword, tag):
       pos = '1.0'
       while True:
           idx = text_widget.search(keyword, pos, END)
           if not idx:
               break
           pos = '{}+{}c'.format(idx, len(keyword))
           text_widget.tag_add(tag, idx, pos)

search(t, 'Failed', 'failed')
#search(t, 'Passed', 'passed')

#t.tag_delete('failed')
#t.tag_delete('passed')

root.mainloop()

In [10]:
import tkinter as tk 
import tkinter.scrolledtext as tst

#v = tk.StringVar()
class Application(tk.Frame):
    def __init__(self, master=None):  
        tk.Frame.__init__(self, master)
        self.grid()
        self.createWidgets()     
        self.createMenu()         
        root['menu'] = self.menubar 
        root.bind('<Button-3>', self.f_popup)
        #root.bind('<Button-1>', self.callback)

         # 状态栏
        self.status = tk.StringVar()
        self.status.set('Ln: 1 Col: 1')
        self.lblStatus = tk.Label(self, textvariable=self.status, anchor='c')
        self.lblStatus.grid(row=7, column=0, columnspan=20, sticky = tk.S + tk.E)
        root.bind('<Key>', self.loc)
        root.bind('<Button-1>', self.loc)
        
    def createWidgets(self):             
        self.textEdit = tst.ScrolledText(self, width=80, height=25)
        self.textEdit.grid(row=0, column=0, rowspan=6)
        
        '''
        self.lblState = tk.Label(self, text="状态栏:")
        self.lblState.grid(row=7, column=0, columnspan=10)
        self.entryState = tk.Entry(self, textvariable=v)
        self.lblState.grid(row=7, column=11, columnspan=20)
        '''
        
    def createMenu(self):              
        self.menubar = tk.Menu(root)   
        #创建子菜单
        self.menufile = tk.Menu(self.menubar)     
        self.menuedit = tk.Menu(self.menubar, tearoff=0) 
        self.menuhelp = tk.Menu(self.menubar, tearoff=0)
        self.menubar.add_cascade(label='File', menu=self.menufile)
        self.menubar.add_cascade(label="Edit", menu=self.menuedit)
        self.menubar.add_cascade(label="Help", menu=self.menuhelp)
        
        #添加菜单项
        self.menufile.add_command(label='New', command=self.f_new)
        self.menufile.bind("<Control-N>", self.f_new)
        self.menufile.add_command(label='Open', command=self.f_open)
        self.menufile.bind("<Control-O>", self.f_open)
        self.menufile.add_command(label='Save', accelerator='^A', 
command=self.f_save)
        self.menufile.bind("<Control-S>", self.f_save)
        #分隔
        self.menufile.add_separator()  
        self.menufile.add_command(label='Exit', command=root.destroy) 
        self.menuedit.add_command(label="Cut", command=self.f_cut, accelerator='Ctrl+T')
        self.menuedit.bind("<Control-T>", self.f_cut)
        self.menuedit.add_command(label="Copy", command=self.f_copy, accelerator='Ctrl+C')
        self.menuedit.bind("<Control-T>", self.f_copy)
        self.menuedit.add_command(label="Paste", command=self.f_paste, accelerator='Ctrl+V')
        self.menuedit.bind("<Control-T>", self.f_paste)
        self.menuedit.add_command(label="Delete", command=self.f_delete, accelerator='Ctrl+D')
        self.menuedit.bind("<Control-T>", self.f_delete)
        self.menuhelp.add_command(label="About", command=self.f_about)
    def f_new(self):
        root.title('untitle')       #新建文件title
        self.textEdit.delete(1.0, tk.END)  
    def f_open(self):           
        self.textEdit.delete(1.0, tk.END) 
        fname = tk.filedialog.askopenfilename(filetypes=[('Python源文件','.py')])
        with open(fname, 'r', encoding= 'utf-8') as f1: 
            str1 = f1.read()                
        self.textEdit.insert(0.0, str1)         
    def f_save(self):          
        str1 = self.textEdit.get(1.0, tk.END)   
        fname = tk.filedialog.asksaveasfilename(filetypes=[('Python源文件','.py')])
        with open(fname, 'w', encoding= 'utf-8') as f1:  #打开文件
            f1.write(str1)      
    def f_about(self):           
        tk.messagebox.showinfo('关于', '版本V 1.0.1')
    def f_cut(self):
        try:
            str1=self.textEdit.get(tk.SEL_FIRST, tk.SEL_LAST)
            self.textEdit.clipboard_clear()        
            self.textEdit.clipboard_append(str1)    
            self.textEdit.delete(tk.SEL_FIRST, tk.SEL_LAST)
        except: pass
    def f_copy(self):           
        try:
            str1=self.textEdit.get(tk.SEL_FIRST, tk.SEL_LAST)
            self.textEdit.clipboard_clear()        
            self.textEdit.clipboard_append(str1)   
        except: pass
    def f_paste(self):          
        str1 = self.textEdit.selection_get(selection='CLIPBOARD')
        try:           
            self.textEdit.replace(tk.SEL_FIRST, tk.SEL_LAST, str1)
        except:
            self.textEdit.insert(tk.INSERT, str1)
    def f_delete(self):          
        try:
            str1=self.textEdit.get(tk.SEL_FIRST, tk.SEL_LAST) 
            self.textEdit.delete(tk.SEL_FIRST, tk.SEL_LAST)
        except: pass
    #定位光标位置，缺点读字符串的位置。。，不知道换行在哪里，仍需加强条件调试。
    def loc(self,event=None):
        s = str(self.textEdit.index('insert')).split('.')
        statusXY = 'Ln: ' + str(s[0]) + 'Col: ' + str(s[1])
        self.status.set(statusXY)

    def f_popup(self, event):   
        self.menuedit.post(event.x_root, event.y_root)
        

root = tk.Tk()                
root.title('文本编辑器')
app = Application(master=root)   
app.mainloop()         

In [29]:
import chardet
chardet.detect(b'Hello, world!')
data = '台積電'.encode('gbk')
chardet.detect(data)

{'encoding': None, 'confidence': 0.0, 'language': None}